## Base API Client Class Authorisation
This notebook builds on getting an authorisation token from Spotify to be able to make requests to their API, and brings all of that code into a single SpotifyAPI class for simpler implementation into projects.

In [1]:
!pip install requests

You should consider upgrading via the '/Users/alexgidman/RTN/SpotifyAPI_nb/env/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import requests
import datetime

In [3]:
import base64

In [6]:
client_id = "a3aadf7f4d184af0af9d849d96e89b12"
client_secret = ""

In [11]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"

    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        """
        Returns a base64 encoded String.
        """
        client_id = self.client_id
        client_secret = self.client_secret
        
        if client_secret == None or client_id == None:
            raise Exception("client_id and client_secret required.")
            
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return { 
        "Authorization" : f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
        "grant_type" : "client_credentials"
        }
        
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200,299):
            return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True

In [16]:
spotify = SpotifyAPI(client_id, client_secret)
spotify.perform_auth()

True

In [17]:
spotify.access_token

'BQC-vsUjmYoe2C0z466JhT0rqHw1MwkwAVwlsGDLoivIO40sdHZUJilMciOQJHoLF9K2rg6Te0vJ0vXncqw'